In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import re
import os
from datetime import datetime, timedelta

# --- 1. 文件路径配置 ---
FILE_PATH = r"D:\5507课件\作业\泡泡玛特_101页起_截止240601.csv"
SAVE_PATH = r"D:\5507课件\作业\泡泡玛特_101页起_清洗倒序版.csv"

# --- 2. 时间基准设置 ---
# 默认年份 (针对只显示 '09-12' 这种数据)
DEFAULT_YEAR = 2025
# 基准日期 (针对 '今天'/'昨天'，通常设为爬取当天的日期)
CURRENT_REF_DATE = datetime(2025, 11, 23)

def clean_text_content(text):
    """
    清洗内容中的乱码
    """
    if not isinstance(text, str):
        return ""
    # 替换乱码符号
    text = text.replace('', '').replace('?', '')
    # 移除不可见字符
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    # 规范空格
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def parse_and_standardize_time(time_str):
    """
    标准化时间格式，用于排序
    """
    if not isinstance(time_str, str) or not time_str.strip():
        return pd.NaT
    
    time_str = time_str.strip()
    
    try:
        # 情况A: 完整日期 "2024-05-20"
        if re.match(r'^\d{4}-\d{1,2}-\d{1,2}', time_str):
            return pd.to_datetime(time_str)
        
        # 情况B: 无年份 "05-20 12:00" -> 补全为默认年份
        elif re.match(r'^\d{1,2}-\d{1,2}', time_str):
            full_str = f"{DEFAULT_YEAR}-{time_str}"
            return pd.to_datetime(full_str)
            
        # 情况C: "今天 12:00"
        elif "今天" in time_str:
            time_part = re.search(r'\d{1,2}:\d{1,2}', time_str).group()
            full_str = f"{CURRENT_REF_DATE.strftime('%Y-%m-%d')} {time_part}"
            return pd.to_datetime(full_str)
            
        # 情况D: "昨天 12:00"
        elif "昨天" in time_str:
            yesterday = CURRENT_REF_DATE - timedelta(days=1)
            time_part = re.search(r'\d{1,2}:\d{1,2}', time_str).group()
            full_str = f"{yesterday.strftime('%Y-%m-%d')} {time_part}"
            return pd.to_datetime(full_str)
            
        # 情况E: "刚刚", "分钟前"
        elif "分钟前" in time_str or "秒前" in time_str or "刚刚" in time_str:
            return CURRENT_REF_DATE
            
        else:
            return pd.NaT
            
    except:
        return pd.NaT

def main():
    print(f"正在读取文件: {FILE_PATH}")
    if not os.path.exists(FILE_PATH):
        print("错误：找不到该文件。")
        return

    # 读取CSV
    try:
        df = pd.read_csv(FILE_PATH, encoding='utf-8-sig')
    except:
        try:
            df = pd.read_csv(FILE_PATH, encoding='gbk')
        except Exception as e:
            print(f"读取出错: {e}")
            return

    raw_rows = len(df)
    print(f"原始数据行数: {raw_rows}")

    # 1. 清洗内容
    print("正在清洗微博内容乱码...")
    df['微博内容'] = df['微博内容'].apply(clean_text_content)

    # 2. 解析时间
    print("正在标准化时间格式...")
    df['temp_time'] = df['发布时间'].apply(parse_and_standardize_time)

    # 3. 删除无效行
    df_clean = df.dropna(subset=['temp_time']).copy()
    print(f" -> 删除无效时间行: {raw_rows - len(df_clean)} 条")

    # 4. 倒序排列 (最新 -> 最旧)
    print("正在按时间倒序排列...")
    df_clean = df_clean.sort_values(by='temp_time', ascending=False)

    # 5. 整理保存
    # 将标准时间写回 '发布时间' 列
    df_clean['发布时间'] = df_clean['temp_time']
    df_clean = df_clean.drop(columns=['temp_time'])

    print(f"正在保存至: {SAVE_PATH}")
    df_clean.to_csv(SAVE_PATH, index=False, encoding='utf-8-sig')

    print("-" * 40)
    print("清洗完成！")
    print(f"最终有效数据: {len(df_clean)} 条")
    print("-" * 40)

if __name__ == "__main__":
    main()

正在读取文件: D:\5507课件\作业\泡泡玛特_101页起_截止240601.csv
原始数据行数: 2459
正在清洗微博内容乱码...
正在标准化时间格式...
 -> 删除无效时间行: 52 条
正在按时间倒序排列...
正在保存至: D:\5507课件\作业\泡泡玛特_101页起_清洗倒序版.csv
----------------------------------------
清洗完成！
最终有效数据: 2407 条
----------------------------------------
